In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load in the data
df = pd.read_csv(r"../../../data/first_1000_JSON_files.csv", low_memory=True)

# Create a copy of the dataframe to work with
df_copy = df.copy()

C:\Users\hb888\AppData\Local\Temp\ipykernel_29888\904903100.py:2: DtypeWarning: Columns (51,59,62,63,67,69,72,73,86,88,90,91,93,94,96,97) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"../../../data/first_1000_JSON_files.csv", low_memory=True)


In [4]:
df.describe()

,imei,tracker.metric.bbatv,tracker.metric.bbatp,tracker.loc.hdop,tracker.loc.alt,tracker.loc.ang,tracker.loc.sp,tracker.gsm.mcc,tracker.gsm.mnc,tracker.gsm.lac,...,event.meta.motion_time,event.meta.post_crash_deviation,event.meta.post_crash_speed,event.meta.pre_crash_deviation,event.meta.probability_threshold,event.meta.start_crash_proba,event.meta.true_crash_proba,event.meta.geofence_id,tracker.config.fwver,event.meta.step
count,7.258610e+05,354249.000000,354249.000000,301817.000000,301817.000000,301817.000000,301817.000000,302377.000000,302377.000000,302377.000000,...,2698.000000,2698.000000,2429.000000,2698.000000,2698.0,2698.0,2698.000000,686.000000,2.00000,6.000000
mean,8.685001e+14,4.026312,84.291834,0.914862,8.728253,132.922890,11.949913,204.014928,8.014664,3283.314981,...,522.137139,0.597368,15.573075,0.608667,50.0,30.0,34.704107,71963.383382,1.55500,2.333333
std,5.889153e+03,0.268007,24.887891,0.385368,75.580734,119.772451,10.108115,0.513157,0.784375,719.523749,...,2891.755068,0.354705,7.356599,0.399790,0.0,0.0,4.838542,14385.329196,0.72832,0.816497
min,8.685001e+14,0.000000,0.000000,0.000000,-644.000000,0.000000,0.000000,204.000000,1.000000,12.000000,...,-1.000000,0.003371,0.000000,0.022666,50.0,30.0,21.870000,54987.000000,1.04000,1.000000
25%,8.685001e+14,3.991000,80.000000,1.000000,-5.000000,0.000000,0.000000,204.000000,8.000000,3140.000000,...,74.000000,0.332655,12.000000,0.313812,50.0,30.0,35.640000,62922.000000,1.29750,2.000000
50%,8.685001e+14,4.102000,97.000000,1.000000,4.000000,114.000000,14.000000,204.000000,8.000000,3260.000000,...,185.000000,0.528799,17.000000,0.494587,50.0,30.0,35.640000,65321.000000,1.55500,2.500000
75%,8.685001e+14,4.147000,100.000000,1.000000,17.000000,240.000000,20.000000,204.000000,8.000000,3360.000000,...,473.750000,0.779753,21.000000,0.798071,50.0,30.0,35.640000,79529.250000,1.81250,3.000000
max,8.685001e+14,4.228000,100.000000,6.000000,9294.000000,359.000000,149.000000,270.000000,99.000000,49308.000000,...,79777.000000,3.334789,37.000000,2.551362,50.0,30.0,85.536000,110384.000000,2.07000,3.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 725861 entries, 0 to 725860
Data columns (total 99 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   _id                                  725861 non-null  object 
 1   imei                                 725861 non-null  float64
 2   dts                                  725861 non-null  object 
 3   dtd                                  699926 non-null  object 
 4   event.key                            190767 non-null  object 
 5   event.dte                            190767 non-null  object 
 6   tracker.metric.bbatv                 354249 non-null  float64
 7   tracker.metric.bbatp                 354249 non-null  float64
 8   tracker.loc.dtg                      301755 non-null  object 
 9   tracker.loc.hdop                     301817 non-null  float64
 10  tracker.loc.alt                      301817 non-null  float64
 11  tracker.loc.a

In [17]:
# show columns that start with device.metric
device_metric_cols = [col for col in df.columns if col.startswith('event.meta')]
print(device_metric_cols)

# show total number of columns that start with device.metric
len(device_metric_cols)

['event.meta.status', 'event.meta.started_at', 'event.meta.angle_threshold', 'event.meta.angular_change', 'event.meta.crash_timeout', 'event.meta.high_deviation_threshold', 'event.meta.low_deviation_threshold', 'event.meta.maximum_speed', 'event.meta.min_seconds_after_crash', 'event.meta.motion_time', 'event.meta.post_crash_deviation', 'event.meta.post_crash_speed', 'event.meta.pre_crash_deviation', 'event.meta.probability_threshold', 'event.meta.ride_event', 'event.meta.start_crash_proba', 'event.meta.true_crash', 'event.meta.true_crash_proba', 'event.meta.uuid', 'event.meta.geofence_id', 'event.meta.triggered_by', 'event.meta.message', 'event.meta.step']


23

In [10]:
# show columns that start with device.config
device_config_cols = [col for col in df.columns if col.startswith('device.config')]
print(device_config_cols)

# show total number of columns that start with device.config
len(device_config_cols)

['device.config.bfcc', 'device.config.bfccp', 'device.config.dwheel', 'device.config.bdcc', 'device.config.bnomv', 'device.config.ecutype', 'device.config.dcontver', 'device.config.dcontser', 'device.config.ddisver', 'device.config.bser', 'device.config.bswver', 'device.config.ddisser', 'device.config.ddcser', 'device.config.ddcver', 'device.config.dpodser', 'device.config.dpodver']


16

In [9]:
df.columns

Index(['_id', 'imei', 'dts', 'dtd', 'event.key', 'event.dte',
       'tracker.metric.bbatv', 'tracker.metric.bbatp', 'tracker.loc.dtg',
       'tracker.loc.hdop', 'tracker.loc.alt', 'tracker.loc.ang',
       'tracker.loc.sp', 'tracker.gsm.mcc', 'tracker.gsm.mnc',
       'tracker.gsm.lac', 'tracker.gsm.cid', 'tracker.metric.rssi',
       'tracker.metric.moving', 'device.metric.deculock',
       'device.metric.dstatus', 'tracker.config.poutput',
       'device.metric.bsocp', 'device.metric.bmv', 'device.config.bfcc',
       'device.config.bfccp', 'device.metric.btemp', 'device.metric.bpackv',
       'tracker.config.ecutype', 'device.metric.bcyc', 'device.config.dwheel',
       'device.config.bdcc', 'device.config.bnomv', 'device.config.ecutype',
       'device.metric.delectemp', 'device.metric.dpedcad',
       'device.metric.dactualsp', 'device.metric.dlight',
       'device.metric.dwheels', 'device.metric.bcur', 'device.metric.bstate',
       'device.metric.msupp', 'device.metric.mpow',

In [25]:
# show rows where device.meta.dodo'].notnull()] and show that column
df[df['device.metric.dtrip'].notnull()]['device.metric.dtrip'].value_counts()

813.1     6269
302.9     5501
1196.8    3896
308.8     3882
71.4      3844
          ... 
2718.4       1
2718.2       1
2718.0       1
2717.7       1
871.2        1
Name: device.metric.dtrip, Length: 42768, dtype: int64

In [24]:
# show rows where device.metric.dodom is not null and show that column
df[df['device.metric.dodom'].notnull()]['device.metric.dodom'].value_counts()

1289000.0    5940
2452000.0    5255
2458000.0    3726
4057000.0    3711
2474000.0    3613
             ... 
924000.0        1
937000.0        1
7222000.0       1
8904000.0       1
1783000.0       1
Name: device.metric.dodom, Length: 8372, dtype: int64

In [ ]:
# show the unique values in the 'event.key' column and their counts
df['event.key'].value_counts()

In [ ]:
# show the entire value in the 'imei' column. not the truncated version
pd.set_option('display.max_colwidth', None)

In [ ]:
df_copy.head(5)

In [ ]:
df.describe()

In [ ]:
# show a few values in the 'imei" column and sohw the entire value not the truncated version
df['imei'].value_counts()

# change the value in the imei column to display  from scientific notation to standard notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df.dtypes

# change imei column to a int64 data type
df['imei'] = df['imei'].astype('int64')

# show the duplicates in the imei column
df['imei'].value_counts()

# show the top 10
df['imei'].value_counts().head(10)

In [ ]:
df_copy.info()

In [ ]:
# Show the columns where the number of NaN values is at least 80% of the rows
df_copy.columns[df_copy.isna().sum() > 0.8 * len(df_copy)]

# Save these columns to a variable called columns_to_drop_because_of_80_percent_NaN
columns_to_drop_because_of_80_percent_NaN = df_copy.columns[df_copy.isna().sum() > 0.8 * len(df_copy)]

# Drop these columns from the dataframe
df_copy = df_copy.drop(columns=columns_to_drop_because_of_80_percent_NaN)

In [ ]:
df_copy

In [ ]:
# Drop the columns that are not useful for the analysis
columns_to_drop = ['_id', 'imei', 'dts', 'dtd', 'event.dte', 'tracker.loc.dtg', 'tracker.loc.hdop', 'tracker.loc.alt', 'tracker.loc.ang', 'tracker.loc.sp', 'tracker.gsm.mcc', 'tracker.gsm.mnc', 'tracker.gsm.lac', 'tracker.gsm.cid',
                   'tracker.metric.deculock', 'tracker.metric.dstatus', 'file_name']

# Check if the columns are in the dataframe and remove the ones that are
for column in columns_to_drop:
    if column in df_copy.columns:
        df_copy = df_copy.drop(columns=column)

In [ ]:
df_copy

In [ ]:
# Show the number of NaN values per column and order them from highest to lowest
df_copy.isna().sum().sort_values(ascending=False)

In [ ]:
# Show the percentage of NaN values per column and order them from highest to lowest
df_copy.isna().sum().sort_values(ascending=False) / len(df_copy)

In [ ]:
# Remove the rows where the number of NaN values is at least 50% of the columns
df_copy = df_copy.dropna(thresh=0.5*len(df_copy.columns))

In [ ]:
# one hot encode the column "event.key" and drop the original column
df_copy = pd.get_dummies(df_copy, columns=['event.key'], drop_first=True)

In [ ]:
df_copy

In [ ]:
# Show the amount of naN vales per column
df_copy.isna().sum()

# Remove the columns with more than 100000 NaN values
df_copy = df_copy.dropna(axis=1, thresh=100000)

df_copy

# Create a dataframe called df_variables that is a copy of df_copy
df_variables = df_copy.copy()

In [ ]:
df_variables

In [ ]:
df_variables.info()

In [ ]:
# for the NaN values, fill them with the mean of the column
df_variables = df_variables.fillna(df_variables.mean())

In [ ]:
# Create a histogram for all the numerical columns
df_variables.hist(figsize=(40, 40))
plt.show()

In [ ]:
# create a correlation matrix
correlation_matrix = df_variables.corr()
correlation_matrix

In [ ]:
# create boplots for the numerical columns, to see if there are any outliers
for column in df_variables.columns:
    if df_variables[column].dtype in ['float64', 'int64']:
        plt.boxplot(df_variables[column])
        plt.title(column)
        plt.show()

        # show outliers
        Q1 = df_variables[column].quantile(0.25)
        Q3 = df_variables[column].quantile(0.75)
        IQR = Q3 - Q1
        print(f"Outliers for {column}:")
        print(df_variables[(df_variables[column] < (Q1 - 1.5 * IQR)) | (df_variables[column] > (Q3 + 1.5 * IQR))][column])
        

In [ ]:
# use a kmeans clustering algorithm to cluster the data and see if there are any patterns, visualise the clusters
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# scale the data
scaler = StandardScaler()

df_variables_scaled = scaler.fit_transform(df_variables)

# create a kmeans model
kmeans = KMeans(n_clusters=50, random_state=0)

# fit the model
kmeans.fit(df_variables_scaled)

# create a new column in the dataframe called "cluster" and assign the cluster to each row
df_variables['cluster'] = kmeans.labels_

# create a scatter plot of the clusters
plt.scatter(df_variables['device.metric.btemp'], df_variables['device.metric.bpackv'], c=df_variables['cluster'])
plt.xlabel('temperature')
plt.ylabel('voltage')
plt.title('Clusters of locations')
plt.show()

In [ ]:
df_variables

In [ ]:
# visualize the outliers
plt.scatter(df_variables['device.metric.btemp'], df_variables['device.metric.bpackv'])
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.title('Outliers of locations')
plt.show()